<a href="https://www.kaggle.com/code/prisha09/glaucoma-detection?scriptVersionId=224679999" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.activations import relu, sigmoid
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from skimage import morphology

import matplotlib.pyplot as plt

IMAGES_FOLDER = "/kaggle/input/glaucoma-datasets/G1020/Images"
df = pd.read_csv("/kaggle/input/glaucoma-datasets/G1020/G1020.csv")
MASK_FOLDER = "/kaggle/input/glaucoma-datasets/G1020/Masks"

df = df.sample(n=150, random_state=42)
df = df.head(150)

def load_data(df, image_size=(512, 512)):
    images, labels, od_masks, oc_masks = [], [], [], []
    
    print(f"Total images in dataframe: {len(df)}")

    if not os.path.exists(IMAGES_FOLDER):
        print(f"Error: IMAGES_FOLDER not found: {IMAGES_FOLDER}")
    else:
        print(f"IMAGES_FOLDER found: {IMAGES_FOLDER}")

    missing_images = [img for img in df['imageID'] if not os.path.exists(os.path.join(IMAGES_FOLDER, img))]
    if missing_images:
        print(f"Missing images: {len(missing_images)} images not found.")

    def rotate_image(image, angle):
        h, w = image.shape[:2]
        center = (w // 2, h // 2)
        rot_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(image, rot_matrix, (w, h))
        return cv2.resize(rotated, (512, 512)).reshape(512, 512, 1)

    for index, row in df.iterrows():
        img_name = row['imageID']
        label = row['binaryLabels']
        img_path = os.path.join(IMAGES_FOLDER, img_name)

        base_name = os.path.splitext(img_name)[0]  
        mask_name = f"{base_name}.png" 
        mask_path = os.path.join(MASK_FOLDER, mask_name)

        if os.path.exists(img_path) and os.path.exists(mask_path):
            image = cv2.imread(img_path, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, image_size)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.GaussianBlur(image, (65, 65), 0)
            image = image.reshape(image_size[0], image_size[1], 1)

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, image_size)

            optic_disc = np.where(mask == 1, 255, 0).astype(np.uint8).reshape(image_size[0], image_size[1], 1)
            optic_cup = np.where(mask == 2, 255, 0).astype(np.uint8).reshape(image_size[0], image_size[1], 1)

            images.append(image)
            labels.append(label)
            od_masks.append(optic_disc)
            oc_masks.append(optic_cup)

            for angle in [45, 90, 170]:
                rotated_img = rotate_image(image, angle)
                rotated_od = rotate_image(optic_disc, angle)
                rotated_oc = rotate_image(optic_cup, angle)

                images.append(rotated_img)
                labels.append(label)
                od_masks.append(rotated_od)
                oc_masks.append(rotated_oc)

        else:
            print(f"Image not found: {img_path}")

    images = np.array(images) / 255.0
    labels = np.array(labels)
    od_masks = np.array(od_masks) / 255.0
    oc_masks = np.array(oc_masks) / 255.0

    print(f"Total images after augmentation: {len(images)}")
    print(f"Total labels: {len(labels)}")
    print(f"Total OD masks: {len(od_masks)}")
    print(f"Total OC masks: {len(oc_masks)}")

    return images, labels, od_masks, oc_masks  
    
images, labels, od_masks, oc_masks = load_data(df)

X_train, X_temp, y_train, y_temp, od_train, od_temp, oc_train, oc_temp = train_test_split(
    images, labels, od_masks, oc_masks, test_size=0.3, random_state=42)
    
X_val, X_test, y_val, y_test, od_val, od_test, oc_val, oc_test = train_test_split(
    X_temp, y_temp, od_temp, oc_temp, test_size=0.5, random_state=42)


def build_unet():
    inputs = layers.Input((512, 512, 1))

    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    c4 = Dropout(0.5)(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Dropout(0.5)(c5)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    od_outputs = Conv2D(1, (1, 1), activation='sigmoid', name='od_output')(c9)
    oc_outputs = Conv2D(1, (1, 1), activation='sigmoid', name='oc_output')(c9)

    model = Model(inputs=inputs, outputs=[od_outputs, oc_outputs])
    return model

unet = build_unet()
unet.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.0004, momentum=0.95),
    loss='binary_crossentropy',
    metrics=[['accuracy'],['accuracy']]
)

unet.fit(X_train, [od_train, oc_train], epochs=70, batch_size=2, 
         validation_data=(X_val, [od_val, oc_val]))

od_pred, oc_pred = unet.predict(X_test)

print("Prediction complete!")



Total images in dataframe: 150
IMAGES_FOLDER found: /kaggle/input/glaucoma-datasets/G1020/Images
Total images after augmentation: 600
Total labels: 600
Total OD masks: 600
Total OC masks: 600
Epoch 1/70
210/210 ━━━━━━━━━━━━━━━━━━━━ 83s 277ms/step - loss: 1.0685 - oc_output_accuracy: 0.8280 - oc_output_loss: 0.5394 - od_output_accuracy: 0.8844 - od_output_loss: 0.5291 - val_loss: 0.1329 - val_oc_output_accuracy: 0.9962 - val_oc_output_loss: 0.0328 - val_od_output_accuracy: 0.9833 - val_od_output_loss: 0.1001
Epoch 2/70
210/210 ━━━━━━━━━━━━━━━━━━━━ 47s 223ms/step - loss: 0.1249 - oc_output_accuracy: 0.9967 - oc_output_loss: 0.0285 - od_output_accuracy: 0.9836 - od_output_loss: 0.0964 - val_loss: 0.1236 - val_oc_output_accuracy: 0.9962 - val_oc_output_loss: 0.0298 - val_od_output_accuracy: 0.9833 - val_od_output_loss: 0.0938
Epoch 3/70
210/210 ━━━━━━━━━━━━━━━━━━━━ 47s 223ms/step - loss: 0.1208 - oc_output_accuracy: 0.9965 - oc_output_loss: 0.0271 - od_output_accuracy: 0.9831 - od_output_l

In [2]:
def calculate_cdr(disc_mask, cup_mask):
    disc_area = np.sum(disc_mask)
    cup_area = np.sum(cup_mask)
    disc_mask = (disc_mask > 0).astype(np.uint8)
    cup_mask = (cup_mask > 0).astype(np.uint8)


    return round((2 * cup_area) / disc_area, 4) if disc_area != 0 else 0

def extract_isnt_quadrants(disc_mask, cup_mask):
    disc_mask = disc_mask.squeeze()
    cup_mask = cup_mask.squeeze()

    disc_mask_rotated = np.rot90(disc_mask)
    cup_mask_rotated = np.rot90(cup_mask)

    nrr_mask = cv2.bitwise_xor(disc_mask_rotated, cup_mask_rotated)

    height, width = disc_mask.shape
    I = np.sum(nrr_mask[height//2:, :])
    S = np.sum(nrr_mask[:height//2, :])
    N = np.sum(nrr_mask[:, :width//2])
    T = np.sum(nrr_mask[:, width//2:])

    return round((1 + (I + S)) / (1 + (N + T)), 4) if (N + T) != 0 else 0

def extract_blood_vessels(fundus_image):
    if len(fundus_image.shape) == 2 or fundus_image.shape[2] == 1:
        green_channel = fundus_image[:, :, 0].astype(np.uint8) 
    else:
        green_channel = fundus_image[:, :, 1].astype(np.uint8) 
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(6,6))
    enhanced = clahe.apply(green_channel)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    bottom_hat = cv2.morphologyEx(enhanced, cv2.MORPH_BLACKHAT, kernel)

    threshold_value = max(10, 3.15 * np.std(bottom_hat))
    _, vessel_mask = cv2.threshold(bottom_hat, threshold_value, 255, cv2.THRESH_BINARY)

    height, width = vessel_mask.shape
    I = np.sum(vessel_mask[height//2:, :])
    S = np.sum(vessel_mask[:height//2, :])
    N = np.sum(vessel_mask[:, :width//2])
    T = np.sum(vessel_mask[:, width//2:])

    return round((1 + (I + S)) / (1 + (N + T)), 4) if (N + T) != 0 else 0

cdr_train = np.array([calculate_cdr(od, oc) for od, oc in zip(od_train, oc_train)])
isnt_train = np.array([extract_isnt_quadrants(od, oc) for od, oc in zip(od_train, oc_train)])
bvr_train = np.array([extract_blood_vessels(img) for img in X_train])

X_train_combined = np.hstack((cdr_train.reshape(-1,1), isnt_train.reshape(-1,1), bvr_train.reshape(-1,1)))

cdr_test = np.array([calculate_cdr(od, oc) for od, oc in zip(od_test, oc_test)])
isnt_test = np.array([extract_isnt_quadrants(od, oc) for od, oc in zip(od_test, oc_test)])
bvr_test = np.array([extract_blood_vessels(img) for img in X_test])

X_test_combined = np.hstack((cdr_test.reshape(-1,1), isnt_test.reshape(-1,1), bvr_test.reshape(-1,1)))


print(f"X_train_combined shape: {X_train_combined.shape}, y_train shape: {y_train.shape}")
print(f"X_test_combined shape: {X_test_combined.shape}, y_test shape: {y_test.shape}")

svm = SVC(C=10, gamma=1, kernel='rbf', probability=True)
svm.fit(X_train_combined, y_train)
svm_predictions = svm.predict(X_test_combined)

mlp = Sequential([
    Input(shape=(X_train_combined.shape[1],)), 
    Dense(3, activation='relu'),
    Dense(2, activation='relu'),  
    Dense(1, activation='sigmoid')  
])
mlp.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy'])
mlp.fit(X_train_combined, y_train, epochs=120, batch_size=2, verbose=1)
mlp_predictions = (mlp.predict(X_test_combined) > 0.5).astype(int).flatten()

adaboost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3), n_estimators=100, random_state = 10)
adaboost.fit(X_train_combined, y_train)
adaboost_predictions = adaboost.predict(X_test_combined)

svm_accuracy = accuracy_score(y_test, svm_predictions)
mlp_accuracy = accuracy_score(y_test, mlp_predictions)
adaboost_accuracy = accuracy_score(y_test, adaboost_predictions)
print(f"SVM Accuracy: {svm_accuracy}")
print(f"MLP Accuracy: {mlp_accuracy}")
print(f"AdaBoost Accuracy: {adaboost_accuracy}")

X_train_combined shape: (420, 3), y_train shape: (420,)
X_test_combined shape: (90, 3), y_test shape: (90,)
Epoch 1/120
210/210 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.3805 - loss: 0.7340
Epoch 2/120
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6259 - loss: 0.6973
Epoch 3/120
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6361 - loss: 0.6758
Epoch 4/120
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6346 - loss: 0.6680
Epoch 5/120
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6676 - loss: 0.6546
Epoch 6/120
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6748 - loss: 0.6482
Epoch 7/120
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6349 - loss: 0.6608
Epoch 8/120
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6567 - loss: 0.6500
Epoch 9/120
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6599 - loss: 0.6464
Epoch 10/120
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6810 - loss: 0.6344
Epoch 11/120
210/210 

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
